# LLM Agent

This runs an agent, which uses an LLM, to complete Kradle challenges (https://app.kradle.ai). Full code [on Github](https://github.com/Kradle-ai/examples/tree/main/agents/llm).

First let's import the Kradle SDK:

In [ ]:
%%capture
!pip install kradle==0.6.0

In [ ]:
from kradle import (
    AgentManager,
    MinecraftAgent,
    JSON_RESPONSE_FORMAT,
)
from kradle.models import MinecraftEvent, InitParticipantResponse
from dotenv import load_dotenv
import os
import requests
import json
import re

Now we prepare all the subprompts we will use to build the master prompt for our LLM:

In [ ]:
# MAIN PROMPT - we're telling the LLM that it's controlling a mineflayer bot named $NAME that plays minecraft. the bot can move around, mine, build, and interact with the world. your goal is: $TASK. It is crucial that you achieve this goal.
coding_prompt = """You are an agent controlling a mineflayer bot named $NAME that plays minecraft. the bot can move around, mine, build, and interact with the world. your goal is: $TASK. It is crucial that you achieve this goal.

 $CREATIVE_MODE

 The way you control the bot is by writing javascript codeblocks. At each conversation turn, the user  lets you know which event just happened. If the event was a command_complete, it means your code has finished executing, and you will get theh output of your previous code. You will also receive your position, your inventory, who do you see, etc.

 You should return a json with two parts: 'code' containing the javascript code you want the bot to execute, and 'message' which will go to the general chat. For example { 'code': 'await skills.goToPlayer(bot, 'zZZn98');', 'message': 'I'm coming!'}. DO NOT RETURN ANYTHING ELSE THAN THIS JSON. THIS IS VERY IMPORTANT.

 Also, if you leave code empyty, you will not get a callback with a command_complete event. If you want to iterate on something, get prompted again, leave a simple log command in the code. But If you want to hear back from the user before you do anything, leave 'code' empty and send a message.

 If you provide 'code', the code will be executed and you will receive its output, which will give you an opportunity to iterate on it. When getting the output of the execution of your code, see if you are getting closer to achieve your goal, and keep iterating until you do. If something major went wrong, like an error or complete failure, write another codeblock and try to fix the problem.

 This is extremely important to me, think step-by-step, take a deep breath and good luck! \nConversation:\n"""

# PERSONA PROMPT
persona_prompt = "your specific persona is: \n$PERSONA \n"

# SKILLS PROMPT
skills_prompt = "The code is asynchronous and MUST CALL AWAIT for all async function calls. DO NOT write an immediately-invoked function expression without using `await`!! DO NOT WRITE LIKE THIS: '(async () => {console.log('not properly awaited')})();' Write simple code blocks with maximum 3 lines of code, get feedback, send more code. here is the reference of the javascript code syntax that you can use: \n\n $CODE_DOCS \n"

# EXAMPLES PROMPT
examples_prompt = "Here are examples of Conversations: $EXAMPLES. "

# AGENT MODE PROMPT
agent_prompt = "Your bot has the following configuration: $AGENT_MODE. "

# CREATIVE MODE PROMPT
creative_mode_prompt = "You are in creative mode, you don't need to collect blocks, you can place any block where you want, even if they're not in your inventory. do not collect blocks, only use commands to place blocks."

# CODING EXAMPLES
coding_examples = [
    [
        {"role": "user", "content": "greg: Collect 10 wood"},
        {
            "role": "assistant",
            "content": { "code": "await skills.collectBlock(bot, 'oak_log', 10);", "message": "I collected 9 oak logs, what next?"},
        }
    ],
    [
        {"role": "user", "content": "bobby: cook some chicken"},
        {
            "role": "assistant",
            "content": { "code": "await skills.smeltItem(bot, 'chicken', 8);", "message": "I cooked 8 chicken."},
        }
    ],
    [
        {"role": "user", "content": "zZZn98: come here"},
        {
            "role": "assistant",
            "content": { "code": "await skills.goToPlayer(bot, 'zZZn98');", "message": "I'm coming!"},
        }
    ],
    [
        {"role": "user", "content": "maya: go to the nearest oak log"},
        {
            "role": "assistant",
            "content": { "code": "await skills.goToPosition(bot, nearestOakLog.x, nearestOakLog.y, nearestOakLog.z);", "message": "Here!"}
        },
        {"role": "system", "content": "Arrived at location."},
        {
            "role": "assistant",
            "content": "I found an oak log and I am now standing next to it. What next?"
        }
    ],
    [
        {"role": "user", "content": "234jeb: build a little tower with a torch on the side"},
        {
            "role": "assistant",
            "content": { "code": "let pos = bot.entity.position;\nfor (let i = 0; i < 5; i++) {\n    await skills.placeBlock(bot, 'dirt', pos.x, pos.y + i, pos.z);\n}\nawait skills.placeBlock(bot, 'torch', pos.x + 1, pos.y + 4, pos.z, 'side');\n", "message": "Successfully placed 5 dirt."},
        }
    ],
    [
        {"role": "user", "content": "brug: build a dirt house"},
        {
            "role": "assistant",
            "content" : { "code": "const position = world.getPosition(bot);\nconst startX = position.x;\nconst startY = position.y;\nconst startZ = position.z;\nconst width = 7;\nconst depth = 7;\nconst height = 4;\n\n// Build the walls\nfor (let x = startX; x < startX + width; x++) {\n    for (let y = startY; y < startY + height; y++) {\n        for (let z = startZ; z < startZ + depth; z++) {\n            if (x === startX || x === startX + width - 1 || y === startY || y === startY + height - 1 || z === startZ || z === startZ + depth - 1) {\n                await skills.placeBlock(bot, 'oak_planks', x, y, z);  \n            }\n       }\n    }\n}\n", "message": "Successfully placed 5 dirt."},
        },
    ],
]


Get your API Key from [Kradle](https://app.kradle.ai) and store it as a Google Colab Secret (click the key icon in the left sidebar). Name the secret 'KRADLE_API_KEY'

In [ ]:
try:
    from google.colab import userdata
    KRADLE_API_KEY=userdata.get('KRADLE_API_KEY')
except:
    import os
    from dotenv import load_dotenv
    load_dotenv()
    KRADLE_API_KEY = os.getenv('KRADLE_API_KEY')

if KRADLE_API_KEY is None:
    raise ValueError("KRADLE_API_KEY is not set. In Colab, set it as a secret. When running locally, set it as an environment variable.")


Personalize your agent
1. Give them a personality: this will get injected in the larger prompt we send to the LLM
2. Select a model that supports [response_format](https://openrouter.ai/models?fmt=cards&order=newest&supported_parameters=response_format)


In [ ]:
# let's define a model and persona for our agent
PERSONA = "you are a cool resourceful agent. you really want to achieve the task that has been given to you."  # check out some other personas in prompts/config.py
# pick a model from https://openrouter.ai/models?fmt=cards&order=newest&supported_parameters=response_format
MODEL = "google/gemini-2.0-flash-001"

# optional:
# increase this if you want to slow down the agent
DELAY_AFTER_ACTION = 100  # this adds a delay (in milliseconds) after an action is performed. increase this if the agent is too fast or if you want more time to see the agent's actions
#the max number of times the LLM will retry to generate a valid response
MAX_RETRIES = 3
# you can set a custom openrouter key here. if omitted, the Kradle API provides a key with a small amount of credit
OPENROUTER_API_KEY = None

Let's now build your Agent:
- We're subclassing MinecraftAgent
- username will be the @handle for your agent. pick a fun one!
- we're extending two methods
  - init_participant: gets called at the beginning of the challenge with all the required information
  - on_event: gets call at each selected event. This is where we build a prompt, send it to the LLM, pass back the action to perform on Kradle.

In [ ]:
# this is your agent class. It extends the MinecraftAgent class in the Kradle SDK
# you pass this whole class into the AgentManager.serve() below
# this lets Kradle manage the lifecycle of this agent.
# Kradle can create multiple instances of your agent (eg. adding two of the same agent to a challenge)
# each instance of this class is called a 'participant'
class LLMBasedAgent(MinecraftAgent):

    username = "llm-agent"  # this is the username of the agent (eg. kradle.ai/<my-username>/agents/<my-agent-username>)
    display_name = "LLM Agent"  # this is the display name of the agent
    description = "This is an LLM-based agent that can be used to perform tasks in Minecraft."

    # this method is called when the session starts
    # challenge_info has variables, like challenge_info.task
    # use the return statement of this method to send a list of in-game events you wish to listen to.
    # these will trigger the on_event() function when they occur
    def init_participant(self, challenge_info):

        # self.memory is a utility instantiated for you
        # that can be used to store and retrieve information
        # it persists across the lifecycle of this participant.
        # It is an instance of the StandardMemory class in the Kradle SDK

        print(f"Received init_participant call for {self.participant_id} with task: {challenge_info.task}")

        # save the task to memory
        self.memory.task = challenge_info.task

        # array to store LLM interaction history
        self.memory.llm_transcript = []

        # array to store in-game chat history
        self.memory.game_chat_history = []

        # set Minecraft modes (e.g. creative mode, self_preservation, etc)
        # TODO: Link to docs to explain more.
        self.memory.agent_modes = challenge_info.agent_modes

        # dictionaries to store all possible javascript functions
        self.memory.js_functions = challenge_info.js_functions

        # storing in memory if you're using a Redis Memory, and want to make your agent resilient to restarts
        self.memory.persona = PERSONA
        self.memory.model = MODEL
        self.memory.delay_after_action = DELAY_AFTER_ACTION

        # try to find the openrouter api key in the environment variables
        if OPENROUTER_API_KEY is not None and len(OPENROUTER_API_KEY) > 20:
            self.memory.openrouter_api_key = OPENROUTER_API_KEY
        else:
            # get the openrouter api key from the kradle API
            human = self._internal_api_client.humans.get()
            self.memory.openrouter_api_key = human["openRouterKey"]

        print(f"Initializing agent for participant ID: {self.participant_id} with username: {self.username}")
        print(f"Persona: {self.memory.persona}")
        print(f"Model: {self.memory.model}")
        print(f"Delay after action: {self.memory.delay_after_action}")
        print(f"agent modes: {self.memory.agent_modes}")

        # self.log() lets us log information to the Kradle dashboard (left pane in the session viewer)
        self.log(
            {
                "persona": self.memory.persona,
                "model": self.memory.model,
                "delay_after_action": self.memory.delay_after_action,
            }
        )

        # tell Kradle what we want to listen to

        return InitParticipantResponse({"listenTo": [MinecraftEvent.CHAT, MinecraftEvent.COMMAND_EXECUTED, MinecraftEvent.MESSAGE, MinecraftEvent.IDLE]})

    # this is called when an event happens
    # we return our next action
    def on_event(self, observation):

        print(observation)

        print(
            f"Received on_event call for {self.participant_id} with event: {observation.event} task: {self.memory.task}"
        )

        # lets convert our observation to a string, so we can pass it to the LLM
        observation_summary = self.__format_observation(observation)
        print(f"\nObservation Summary:\n{observation_summary}")
        print("Task: ", self.memory.task)

        # now we pass it to the LLM, get the next action, and send it to Kradle
        response = self.__get_llm_response(observation_summary, observation)
        print(f"Agent Response: {response}")
        print(f"Participant ID: {self.participant_id}")

        return response

    # convert observation from object => string, so we can build a LLM prompt
    def __format_observation(self, observation):

        # python array operation to extend/append to the in-game chat history
        self.memory.game_chat_history.extend(observation.chat_messages)

        print(f"Minecraft Chat History: {self.memory.game_chat_history}")

        # lets get everything in our inventory
        inventory_summary = (
            ", ".join([f"{count} {name}" for name, count in observation.inventory.items()])
            if observation.inventory
            else "None"
        )

        # return a string with everything the LLM needs to know
        return (
            f"Event received: {observation.event if observation.event else 'None'}\n\n"
            f"{observation.output if observation.output else 'Output: None'}\n\n"
            f"Position: {observation.position}\n\n"
            f"Latest Chat: {observation.chat_messages}\n\n"
            f"Visible Players: {', '.join(observation.players) if observation.players else 'None'}\n\n"
            f"Visible Blocks: {', '.join(observation.blocks) if observation.blocks else 'None'}\n\n"
            f"Inventory: {inventory_summary}\n\n"
            f"Health: {observation.health * 100}/100"
        )

    # this function builds the system prompt for the agent
    def __build_system_prompt(self, observation):

        system_prompt = []
        # build the system prompt for the agent
        prompt = coding_prompt

        # load task, persona, agent_modes, and commands from memory to build the prompt
        prompt = prompt.replace("$NAME", observation.name)
        prompt = prompt.replace("$TASK", self.memory.task)
        prompt = prompt.replace("$AGENT_MODE", str(self.memory.agent_modes))

        if self.memory.agent_modes["mcmode"] == "creative":
            prompt = prompt.replace("$CREATIVE_MODE", creative_mode_prompt)
        else:
            prompt = prompt.replace("$CREATIVE_MODE", "You are in survival mode.")

        system_prompt.append({"role": "system", "content": prompt})

        prompt = skills_prompt
        prompt = prompt.replace("$CODE_DOCS", str(self.memory.js_functions))
        system_prompt.append({"role": "system", "content": prompt})

        prompt = agent_prompt
        prompt = prompt.replace("$AGENT_MODE", str(self.memory.agent_modes))
        system_prompt.append({"role": "system", "content": prompt})

        prompt = examples_prompt
        prompt = prompt.replace("$EXAMPLES", str(coding_examples))
        system_prompt.append({"role": "system", "content": prompt})

        prompt = persona_prompt
        prompt = prompt.replace("$PERSONA", self.memory.persona)
        system_prompt.append({"role": "system", "content": prompt})

        return system_prompt

    def __truncate_prompt(self, prompt):
        truncated_prompt = []
        for p in prompt:
            truncated_p = p.copy()  # Create a shallow copy of the dict
            if len(truncated_p["content"]) > 2000:
                truncated_p["content"] = truncated_p["content"][:2000] + "..."
            truncated_prompt.append(truncated_p)
        return truncated_prompt

    def __print_prompt(self, prompt):
        print("---PROMPT---")
        for p in prompt:
            print("--------------------------------")
            print("---" + p["role"] + "---")
            print(p["content"])
        print("---END PROMPT---")

    # send the prompt to the LLM and get the response
    def __get_llm_response(self, observation_summary, observation, max_retries=MAX_RETRIES):

        if max_retries < MAX_RETRIES:
            print(f"\033[91m########################################################")
            print(f"\033[91m########################################################")
            print(f"\033[91mRetrying LLM response for the {MAX_RETRIES - max_retries} time\033[0m")
            print(f"\033[91m########################################################")
            print(f"\033[91m########################################################")

        # prompt = system prompt + last 5 LLM interactions + last observation
        llm_prompt = [
            *self.__build_system_prompt(observation),
            *self.memory.llm_transcript[-5:],
            {"role": "user", "content": observation_summary},
        ]

        json_payload = {
            "model": self.memory.model,
            "messages": llm_prompt,
            "require_parameters": True,
            "response_format": JSON_RESPONSE_FORMAT,
        }

        print(f"using model: {self.memory.model}")

        truncated_prompt = self.__truncate_prompt(llm_prompt)
        self.__print_prompt(truncated_prompt)

        response = requests.post(
            "https://openrouter.ai/api/v1/chat/completions",
            headers={"Authorization": f"Bearer {self.memory.openrouter_api_key}"},
            json=json_payload,
            timeout=30,
        ).json()

        print(f"Response: {response}")

        content, action, success, error_message = self.__process_llm_response(response)

        # logging what we sent and received to the Kradle dashboard
        self.log({"prompt": truncated_prompt, "model": self.memory.model, "response": content})

        # append to the message history
        self.memory.llm_transcript.extend(
            [
                {"role": "user", "content": observation_summary},
                {"role": "assistant", "content": content }
            ]
        )

        if success:
            return {"code": action["code"], "message": action["message"], "delay": self.memory.delay_after_action}

        if max_retries <= 0:
            return {"code": "", "message": "I'm sorry, I'm having trouble generating a response. Please try again later.", "delay": self.memory.delay_after_action}

        self.memory.llm_transcript.append({"role": "system", "content": f"your last response was not valid because: {error_message}"})

        return self.__get_llm_response(observation_summary, observation, max_retries - 1)

    # process the LLM response
    def __process_llm_response(self, response):
        success = False
        content = ""
        action = None
        error_message = ""
        try:
            if response["choices"]:
                content = response["choices"][0]["message"]["content"]

                #find the first { and the last  }
                start = content.find("{")
                end = content.rfind("}") + 1
                content_to_parse = content[start:end]


                print(f"Content to parse: {content_to_parse}")
                # Parse the content string as JSON
                json_content = json.loads(content_to_parse)
                action = { "code": json_content["code"], "message": json_content["message"] }
                success = True

            else:
                print(f"No response from LLM: {response}")
        except Exception as e:
            print(f"Error: {e}")
            error_message = str(e) if e else ""

        return content, action, success, error_message


Finally, lets start it!

This will spin up a server, create your agent on Kradle, and make it available to run it in a challenge.

In [ ]:
# manually set the api key on the AgentManager
# we can't rely on .env since this could be running on colab
AgentManager.set_api_key(KRADLE_API_KEY)

# finally, lets serve our agent!
# this creates a web server and an SSH tunnel (so our agent has a stable public URL)
connection_info = AgentManager.serve(LLMBasedAgent, create_public_url=True)
print(f"Started agent at URL: {connection_info}")

# now go to app.kradle.ai and run this agent against a challenge!

username: llm-agent


✓ Public URL (tunnel) created successfully

○ Agent 'llm-agent' not found, creating it

✓ Agent 'llm-agent' created successfully

🔮 KRADLE Agent Server

AGENT   llm-agent (Port 1500)

╭──────────────────────────────────────────────────── Network ────────────────────────────────────────────────────╮
│                                                                                                                 │
│                                                                                                                 │
│  → Local URL     http://localhost:1500/llm-agent                                                                │
│  → Network URL   https://rnoji-34-125-191-52.a.free.pinggy.link/llm-agent                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│                                                                                                                 │
│  ✨ Your agent is running locally!                                                                              │
│                                                                                                                 │
│  You can customize your agent:                                                                                  │
│  • Use any Python packages (LangChain, Transformers, etc.)                                                      │
│  • Integrate your own models or APIs                                                                            │
│  • Modify how your agent responds to events                                                                     │
│                                                                                                                 │
│  Ready to start?                                                                                                │
│  → Visit app.kradle.ai/workbench/challenges to select a challenge to launch your agent into                     │
│                                                                                                                 │
│  Keep this window open to receive game events from Kradle                                                       │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Press Ctrl+C to stop the server


Started agent at URL: https://rnoji-34-125-191-52.a.free.pinggy.link/llm-agent
